In [81]:
import numpy as np
import pandas as pd
import requests
import json

In [102]:
###Data fetching from ACS data site using census API

key = '42c599b870399ad2f8cfb7b0ef29ffde645ad83f'
url = f'https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E,B01001_002E,B01001_026E,B01002_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E&for=block%20group:*&in=state:42&in=county:*&in=tract:*&key={key}'

#B01001_001E: Total_Pop
#B01001_002E: Total_Pop_M
#B01001_026E: Total_Pop_F
#B01002_001E: Median_Age
#B02001_002E: Race(White)
#B02001_003E: Race(African)
#B02001_004E: Race(NativeAmerican)
#B02001_005E: Race(Asian)


response = requests.get(url)
response.status_code

census = response.text
census = json.loads(census)
df = pd.DataFrame.from_dict(census)

df.columns = df.iloc[0]
df = df.drop(0)
df = df.drop('NAME', axis=1)

In [83]:
import psycopg2

In [96]:
###Create a table called 'rkuremat_acs_data'
conn = psycopg2.connect("host=acs-db.mlpolicylab.dssg.io dbname=acs_data_loading user=mlpp_student password=CARE-horse-most")

cur = conn.cursor()

cur.execute('''CREATE TABLE rkuremat_acs_data(
    Total_Pop text,
    Total_Pop_M text,
    Total_Pop_F text,
    Median_Age text,
    Race_White text,
    Race_African text,
    Race_NativeAmerican text,
    Race_Asian text,
    state text,
    county text,
    track text,
    block_group text,
    PRIMARY KEY(state, county, track, block_group)
)
''')

conn.commit()

In [92]:
### For dropping the table

#conn = psycopg2.connect("host=acs-db.mlpolicylab.dssg.io dbname=acs_data_loading user=mlpp_student password=CARE-horse-most")
#cur = conn.cursor()
#cur.execute('DROP TABLE rkuremat_acs_data')
#conn.commit()

In [103]:
### Inserting data into 'rkuremat_acs_data' table
df.to_csv('df.csv',index=False)

conn = psycopg2.connect("host=acs-db.mlpolicylab.dssg.io dbname=acs_data_loading user=mlpp_student password=CARE-horse-most")

cur = conn.cursor()

with open('df.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'rkuremat_acs_data', sep=',')

conn.commit()